In [2]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import pandas as pd
import os
import collections
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import f1_score
import warnings
warnings.filterwarnings('ignore')
######### Valence And Arousal
def SVMPCA(UserName,Components=10,Window=256,StartTime=0,EndTime=63):
    file_path = os.path.join("ChannelsExtracted",UserName, 'ArousalEncoded'+".csv")
    ArousalFile =  np.genfromtxt(fname=file_path,delimiter=',').astype(int)
    file_path = os.path.join("ChannelsExtracted",UserName, 'ValenceEncoded'+".csv")
    ValenceFile =  np.genfromtxt(fname=file_path,delimiter=',').astype(int)
    BandsFile = np.genfromtxt('training/'+str(StartTime)+str(EndTime)+'/'+str(Window)+UserName+'.csv',delimiter=',')
    X_train, X_test, y_train, y_test , z_train,z_test= train_test_split(BandsFile, ValenceFile,ArousalFile, test_size=0.2)
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    # PCA to select features
    pca = PCA(n_components=Components, svd_solver='full')
    pca.fit(BandsFile)
    BandsFile = pca.transform(BandsFile)
    #Predict Valence SVM and PCA
    clf = SVC()
    clf.fit(X_train, y_train)
    y_predict = clf.predict(X_test)
    #Predict Arousal SVM and PCA
    # Feature Scaling
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
        
    # PCA to select features
    pca = PCA(n_components=Components, svd_solver='full')
    pca.fit(BandsFile)
    BandsFile = pca.transform(BandsFile)
    clf = SVC()
    clf.fit(X_train, z_train)
    z_predict = clf.predict(X_test)
    return [accuracy_score(z_test, z_predict)*100,accuracy_score(y_test, y_predict)*100
            ,f1_score(z_test, z_predict, average='weighted'),
            f1_score(y_test, y_predict, average='weighted')]

######### Happy And sad ##############
def SVMPCAHS(UserName,Components=10,Window=256,StartTime=0,EndTime=63):
    file_path = os.path.join("ChannelsExtracted",UserName, 'HappyAndSadEncoded'+".csv")
    SurveyFile =  np.genfromtxt(fname=file_path,delimiter=',').astype(int)
    BandsFile = np.genfromtxt('training/'+str(StartTime)+str(EndTime)+'/'+str(Window)+UserName+'HS'+'.csv',delimiter=',')
    X_train, X_test, y_train, y_test = train_test_split(BandsFile, SurveyFile, test_size=0.2)
    while ((np.sum(y_train == 0) == 0) or (np.sum(y_train == 1) == 0)): #Don't Start training unless you have 2 classes.. happens with imbalance
        print("here")
        X_train, X_test, y_train, y_test = train_test_split(BandsFile, SurveyFile, test_size=0.2)
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    # PCA to select features
    pca = PCA(n_components=Components, svd_solver='full')
    pca.fit(BandsFile)
    BandsFile = pca.transform(BandsFile)
    #Predict Valence SVM and PCA
    clf = SVC()
    clf.fit(X_train, y_train)
    y_predict = clf.predict(X_test)
    return [accuracy_score(y_test, y_predict)*100,
            f1_score(y_test, y_predict, average='weighted')]

######### All Emotions##############
def SVMPCACombined(UserName,Components=10,Window=256,StartTime=0,EndTime=63):
    file_path = os.path.join("ChannelsExtracted",UserName, 'ValenceAndArousalEncoded'+".csv")
    SurveyFile =  np.genfromtxt(fname=file_path,delimiter=',').astype(int)
    BandsFile = np.genfromtxt('training/'+str(StartTime)+str(EndTime)+'/'+str(Window)+UserName+'.csv',delimiter=',')
    X_train, X_test, y_train, y_test = train_test_split(BandsFile, SurveyFile, test_size=0.2)
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    # PCA to select features
    pca = PCA(n_components=Components, svd_solver='full')
    pca.fit(BandsFile)
    BandsFile = pca.transform(BandsFile)
    #Predict Valence SVM and PCA
    clf = SVC()
    clf.fit(X_train, y_train)
    y_predict = clf.predict(X_test)
    return [accuracy_score(y_test, y_predict)*100,
            f1_score(y_test, y_predict, average='weighted')]


############## Run Each Method Multiple Times ######################
def Run100(UserName,Components=10,Window=256,StartTime=0,EndTime=63):
    arousal=[]
    valence=[]
    F1_ScoreArousal=[]
    F1_ScoreValence=[]
    for i in range (100):
        svm=SVMPCA(UserName,Components,Window,StartTime,EndTime)
        arousal+=[svm[0]]
        valence+=[svm[1]]
        F1_ScoreArousal+=[svm[2]]
        F1_ScoreValence+=[svm[3]]
    print("Arousal "+str(np.mean(arousal)))
    print(collections.Counter(arousal))
    print("Valence "+str(np.mean(valence)))
    print(collections.Counter(valence))
    return [np.mean(arousal),np.mean(valence),np.mean(F1_ScoreArousal),np.mean(F1_ScoreValence)];

def RunCombined100(UserName,Components=10,Window=256,StartTime=0,EndTime=63):
    Combined=[]
    F1_ALL=[]
    for i in range (100):
        AllEmotions=SVMPCACombined(UserName,Components,Window,StartTime,EndTime)
        Combined+=[AllEmotions[0]]
        F1_ALL+=[AllEmotions[1]]
    print("Array of average All Emotions accurecies for user "+UserName)
    print(collections.Counter(Combined))
    return [np.mean(Combined),np.mean(F1_ALL)];
    
def RunHS100(UserName,Components=10,Window=256,StartTime=0,EndTime=63):
    Combined=[]
    F1_HS=[]
    for i in range (100):
        HS=SVMPCAHS(UserName,Components,Window,StartTime,EndTime)
        Combined+=[HS[0]]
        F1_HS+=[HS[1]]
    print("Array of average Happy Sad accurecies for user "+UserName)
    print(collections.Counter(Combined))
    return [np.mean(Combined),np.mean(F1_HS)];
        

    
##################Main#########################33
if __name__ == "__main__":
    HSAccurecy=0;
    HSF1=0;
    CombinedAccurecy=0;
    CombinedF1=0;
    ValenceAccurecy=0;
    ValenceF1=0;
    ArousalAccurecy=0;
    ArousalF1=0;
    for i in range(1,10):
        print("s0"+str(i))
        #Happy and Sad
        var =RunHS100("s0"+str(i))
        print("Happy Sad")
        print(var)
        HSAccurecy+=var[0]
        HSF1+=var[1]
        #All 4 Emotions
        var1 =RunCombined100("s0"+str(i))
        print("Combined Emotions")
        print(var1)
        CombinedAccurecy+=var1[0]
        CombinedF1+=var1[1]
        #Seperated Valence and Arousal
        var2=Run100("s0"+str(i))
        print("Arousal Valence")
        print(var2)
        ArousalAccurecy+=var2[0]
        ValenceAccurecy+=var2[1]
        ArousalF1+=var2[2]
        ValenceF1+=var2[3]
    for i in range(10,33):
        print("s"+str(i))
        #Happy and Sad
        if not ("s"+str(i) == "s31"):
            var =RunHS100("s"+str(i))
            print("Happy Sad")
            print(var)
            HSAccurecy+=var[0]
            HSF1+=var[1]
        #All 4 Emotions
        var1 =RunCombined100("s"+str(i))
        print("Combined Emotions")
        print(var1)
        CombinedAccurecy+=var1[0]
        CombinedF1+=var1[1]
        #Seperated Valence and Arousal
        var2=Run100("s"+str(i))
        print("Arousal Valence")
        print(var2)
        ArousalAccurecy+=var2[0]
        ValenceAccurecy+=var2[1]
        ArousalF1+=var2[2]
    print("Accurecy happy sad for 31 users "+str(HSAccurecy/31))
    print("F1 happy sad for  31 users "+str(HSF1/31))
    print("Accurecy Combined sad for 32 users"+str(CombinedAccurecy/32))
    print("F1 Combined sad for  32 users "+str(CombinedF1/32))
    print("Accurecy Valence for  32 users "+str(ValenceAccurecy/32))
    print("F1 Valence for 32 users "+str(ValenceF1/32))
    print("Accurecy Arousal for  32 users "+str(ArousalAccurecy/32))
    print("F1 Arousal for  32 users "+str(ArousalF1/32))



s01
Array of average Happy Sad accurecies for user s01
Counter({60.0: 45, 40.0: 32, 20.0: 13, 80.0: 10})
Happy Sad
[50.4, 0.4078968253968254]
Array of average All Emotions accurecies for user s01
Counter({25.0: 32, 37.5: 25, 12.5: 21, 50.0: 16, 0.0: 3, 62.5: 3})
Combined Emotions
[29.875, 0.23596464646464646]
Arousal 58.0
Counter({62.5: 41, 50.0: 23, 37.5: 14, 75.0: 14, 87.5: 5, 25.0: 3})
Valence 33.875
Counter({25.0: 32, 37.5: 27, 50.0: 19, 12.5: 14, 62.5: 6, 75.0: 1, 0.0: 1})
Arousal Valence
[58.0, 33.875, 0.47711496836496836, 0.2733398268398268]
s02
Array of average Happy Sad accurecies for user s02
Counter({80.0: 46, 100.0: 38, 60.0: 14, 40.0: 2})
Happy Sad
[84.0, 0.7746825396825396]
Array of average All Emotions accurecies for user s02
Counter({50.0: 36, 37.5: 31, 62.5: 17, 25.0: 11, 75.0: 3, 12.5: 2})
Combined Emotions
[45.5, 0.3187178585303585]
Arousal 65.125
Counter({62.5: 30, 75.0: 25, 50.0: 17, 87.5: 15, 37.5: 10, 100.0: 2, 25.0: 1})
Valence 69.375
Counter({62.5: 32, 75.0: 30

Array of average All Emotions accurecies for user s15
Counter({37.5: 33, 25.0: 24, 50.0: 22, 62.5: 9, 12.5: 8, 0.0: 3, 75.0: 1})
Combined Emotions
[36.75, 0.30747023809523805]
Arousal 46.375
Counter({50.0: 33, 37.5: 24, 62.5: 16, 25.0: 15, 75.0: 9, 12.5: 3})
Valence 60.625
Counter({62.5: 32, 50.0: 26, 75.0: 18, 87.5: 10, 37.5: 9, 25.0: 4, 100.0: 1})
Arousal Valence
[46.375, 60.625, 0.43118073593073597, 0.5872995892995894]
s16
Array of average Happy Sad accurecies for user s16
Counter({100.0: 28, 80.0: 27, 60.0: 25, 40.0: 17, 20.0: 3})
Happy Sad
[72.0, 0.6748412698412699]
Array of average All Emotions accurecies for user s16
Counter({37.5: 27, 25.0: 25, 50.0: 23, 62.5: 11, 12.5: 9, 0.0: 5})
Combined Emotions
[35.875, 0.30448511904761905]
Arousal 65.0
Counter({75.0: 31, 62.5: 29, 50.0: 17, 87.5: 14, 37.5: 6, 25.0: 2, 12.5: 1})
Valence 61.5
Counter({62.5: 40, 50.0: 24, 75.0: 16, 37.5: 8, 87.5: 7, 100.0: 3, 25.0: 1, 12.5: 1})
Arousal Valence
[65.0, 61.5, 0.6451408036408037, 0.5522932900432

Array of average Happy Sad accurecies for user s29
Counter({66.66666666666666: 31, 83.33333333333334: 31, 50.0: 20, 100.0: 14, 33.33333333333333: 3, 16.666666666666664: 1})
Happy Sad
[71.66666666666666, 0.6554250841750842]
Array of average All Emotions accurecies for user s29
Counter({62.5: 26, 50.0: 22, 37.5: 21, 75.0: 14, 12.5: 5, 25.0: 5, 87.5: 5, 100.0: 1, 0.0: 1})
Combined Emotions
[52.875, 0.420868561993562]
Arousal 69.875
Counter({75.0: 37, 62.5: 24, 87.5: 19, 50.0: 11, 37.5: 4, 100.0: 3, 25.0: 2})
Valence 62.875
Counter({62.5: 32, 75.0: 27, 50.0: 19, 37.5: 11, 87.5: 10, 25.0: 1})
Arousal Valence
[69.875, 62.875, 0.6129695304695304, 0.6044933677433678]
s30
Array of average Happy Sad accurecies for user s30
Counter({71.42857142857143: 36, 57.14285714285714: 27, 85.71428571428571: 26, 42.857142857142854: 7, 28.57142857142857: 2, 100.0: 2})
Happy Sad
[68.99999999999999, 0.590256632256632]
Array of average All Emotions accurecies for user s30
Counter({62.5: 32, 50.0: 26, 75.0: 16, 3